In [1]:
import os
import sys
sys.path.append('../')
sys.path.append('../datasets')
sys.path.append('../models')
import torch
import torchvision
import torch.backends.cudnn as cudnn
import random
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from importlib import reload
from datasets.three_dim_shapes import ThreeDimShapesDataset
from datasets.small_norb import SmallNORBDataset
from datasets.seq_mnist import SequentialMNIST
import models.seqae as seqae
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge
from datasets import so3_data as so3d
from einops import rearrange
from sklearn.metrics import r2_score
import pdb
from einops import rearrange
from utils import evaluations as ev
from utils import notebook_utils as nu

import copy

import csv
import ast
from source import yaml_utils as yu


from tqdm import tqdm

if torch.cuda.is_available():
    device = torch.device('cuda')
    cudnn.deterministic = True
    cudnn.benchmark = True
else:
    device = torch.device('cpu')
    gpu_index = -1


    
from utils import yaml_utils as yu
import yaml
rootpath = '/mnt/nfs-mnj-hot-01/tmp/masomatics/block_diag/'
result_dir = '/mnt/nfs-mnj-hot-01/tmp/masomatics/block_diag/result'

mode = 'so3'
dat_root = f"""/mnt/nfs-mnj-hot-01/tmp/masomatics/block_diag/datasets/{mode}"""

baseline_path = os.path.join(result_dir, '20220615_default_run_mnist')

print(baseline_path)
nu.filter_list('202209', os.listdir(result_dir))

/mnt/nfs-mnj-hot-01/tmp/masomatics/block_diag/result/20220615_default_run_mnist


['20220901_Mstar_inv_reg_cnn_0',
 '20220901_Mstar_orth_0',
 '20220906_latentPredict_0']

In [2]:
projname = 'latentPredict'
setting_name = '_'.join(np.sort(nu.filter_list(projname, os.listdir(result_dir)))[0].split('_')[:-1])
print(setting_name)
nu.filter_list(projname, os.listdir(result_dir))


20220906_latentPredict


['20220906_latentPredict_0']

In [3]:


#projname = 'Mstar_orth_mlp_two' 
targdir = f'''20220906_{projname}_0'''
#jobpath =f'''../jobs/20220829_{projname}'''
jobpath = f'''../jobs/{setting_name}'''
print(jobpath)
print(os.path.exists(jobpath))
#jobpath = os.path.join('../jobs/','_'.join(targdir.split('_')[:-1]))


targpath = os.path.join(result_dir, targdir)
targlist = os.listdir(targpath)

targlist = nu.model_exists(targlist,  targpath)
#print(targlist)

nu.filter_list('reg_obs0', targlist)


../jobs/20220906_latentPredict
True


['reg_latent005_reg_obs0_T3_predictiveTrue',
 'reg_latent2_reg_obs0_T5_predictiveFalse',
 'reg_latent2_reg_obs0_T3_predictiveTrue',
 'reg_latent01_reg_obs0_T3_predictiveTrue',
 'reg_latent01_reg_obs0_T5_predictiveFalse',
 'reg_latent005_reg_obs0_T5_predictiveFalse',
 'reg_latent1_reg_obs0_T3_predictiveTrue',
 'reg_latent005_reg_obs0_T3_predictiveFalse',
 'reg_latent01_reg_obs0_T3_predictiveFalse',
 'reg_latent1_reg_obs0_T5_predictiveTrue',
 'reg_latent01_reg_obs0_T5_predictiveTrue',
 'reg_latent1_reg_obs0_T5_predictiveFalse',
 'reg_latent2_reg_obs0_T5_predictiveTrue',
 'reg_latent005_reg_obs0_T5_predictiveTrue',
 'reg_latent2_reg_obs0_T3_predictiveFalse']

In [4]:
checkdir = 'reg_latent005_reg_obs0_T3_predictiveFalse'

In [5]:
reload(ev)
results = {} 
inferred_Ms = {} 
model_configs = {}
models = {}
tp = 1
n_cond = 2
device =  0

Mlist = [] 
targdir_path = os.path.join(targpath, checkdir)

config = nu.load_config(targdir_path)

dataconfig = config['train_data']
dataconfig['args']['T'] = tp + n_cond

data = yu.load_component(dataconfig)
train_loader = DataLoader(data, 
                          batch_size=config['batchsize'],
                          shuffle=True,
                          num_workers=config['num_workers'])

model_config = config['model']
model = yu.load_component(model_config)
iterlist = nu.iter_list(targdir_path)

maxiter = np.max(nu.iter_list(targdir_path))
nu.load_model(model, targdir_path, maxiter)
model = model.eval().to(device)

images = iter(train_loader).next()

# Initialize lazy modules
if type(images) == list:
    images = torch.stack(images)
    images = images.transpose(1, 0)

images = images.to(device)
regconfig = config['reg']


/usr/local/lib/python3.8/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)
/usr/local/lib/python3.8/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


/mnt/nfs-mnj-hot-01/tmp/masomatics/block_diag/result/20220906_latentPredict_0/reg_latent005_reg_obs0_T3_predictiveFalse 
 snapshot_model_iter_100000


In [6]:
T_cond = config['T_cond']
loss,  loss_dict = model.loss(images,  T_cond=T_cond, return_reg_loss=True, reconst=True, regconfig=regconfig)


In [7]:
n_rolls = images.shape[1] - T_cond
H_preds, H_target, fn, M, H = model.compute_H_preds(images, n_rolls=n_rolls, T_cond=T_cond, reconst=True, verbose=True)


In [8]:
loss_dict

{'reg_bd': tensor([0], device='cuda:0'),
 'reg_orth': tensor([0], device='cuda:0'),
 'reg_comm': tensor([0], device='cuda:0'),
 'reg_inv': tensor([0], device='cuda:0'),
 'reg_latent': tensor(0.2729, device='cuda:0', grad_fn=<SumBackward0>),
 'reg_obs': tensor(2393.1384, device='cuda:0', grad_fn=<SumBackward0>)}

In [9]:
H_preds.shape

torch.Size([32, 3, 256, 16])

In [10]:
H.shape

torch.Size([32, 2, 256, 16])

In [11]:
torch.sum((H_target[:, :2]- H)**2)

tensor(0., device='cuda:0', grad_fn=<SumBackward0>)

In [12]:
reload(nu)
nu.l2dist(torch.sigmoid(model.decode(model.encode(images))) , images)

tensor(74.6786, device='cuda:0', grad_fn=<SumBackward0>)

In [13]:
torch.mean( torch.sum((torch.sigmoid(model.decode(H_preds)) - images)**2, axis=[2,3,4])).to('cpu').detach()

tensor(24.9285)

In [14]:
torch.mean( torch.sum((torch.sigmoid(model.decode(model.encode(images))) - images)**2, axis=[2,3,4])).to('cpu').detach()

tensor(0.7779)

In [16]:
nu.l2dist(model.encode(images).to('cpu'), H_preds.to('cpu'))

tensor(0.2729, grad_fn=<SumBackward0>)